In [15]:
# Skriptas išsiųstas Povilui 2022 10 28 bandymams su ištisiniu EKG segmentų su skirtingais pūpsniais išskyrimui. 
# 
#  Skriptas paciento EKG įrašų ir EKG atributų (rpeaks: "sampleIndex" ir "annotationValue") nuskaitymui
# 
# Paciento EKG įrašas koduojamas kodu SUbjCode.
# zive atveju: SubjCode = userNr + recordingNr, kur userNr >= 1000,
#     pvz. SubjCode = 10001
# mit2zive atveju: SubjCode = userNr,  kur userNr < 1000,
#     pvz. SubjCode = 101

import pandas as pd
import numpy as np
from pathlib import Path
import sys, json

def read_rec(rec_dir, SubjCode):
    file_path = Path(rec_dir, str(SubjCode) + '.npy')
    signal = np.load(file_path, mmap_mode='r')
    # print(f"SubjCode: {SubjCode}  signal.shape: {signal.shape}")
    return signal

def read_rec_attrib(rec_dir, SubjCode):
    # Pritaikyta nuskaityti json informaciją tiek mit2zive, tiek zive atvejams
    file_path = Path(rec_dir, str(SubjCode) + '.json')

    if (SubjCode > 1000): # zive atvejis
        with open(file_path,'r', encoding='UTF-8', errors = 'ignore') as f:
            data = json.loads(f.read())
        df = pd.json_normalize(data, record_path =['rpeaks'])
    else: # mit2zive atvejis
        df = pd.read_json(file_path, orient = 'records')

    atr_sample = df['sampleIndex'].to_numpy()
    atr_symbol = df['annotationValue'].to_numpy()
    return atr_sample, atr_symbol

def split_SubjCode(SubjCode):
    """
    zive atveju: SubjCode = int(str(userNr) + str(registrationNr)), kur userNr >= 1000,
    pvz. SubjCode = 10001
    mit2zive atveju: SubjCode = userNr,  kur userNr < 1000,
    pvz. SubjCode = 101

    Parameters
    ------------
        SubjCode: int
    Return
    -----------
        userNr: int
        recordingNr: int
    """   
    if (SubjCode < 1000):
        userNr = SubjCode
        recordingNr = 0   
        return userNr, recordingNr
    else:        
        str_code = str(SubjCode) 
        chars = list(str_code)
        str1 =""
        userNr = int(str1.join(chars[:4]))
        str2 =""
        recordingNr = int(str2.join(chars[4:]))
        return userNr, recordingNr

# Pasiruošimas

my_os=sys.platform
print("OS in my system : ",my_os)

if my_os != 'linux':
    OS = 'Windows'
else:  
    OS = 'Ubuntu'

# Bendras duomenų aplankas

if OS == 'Windows':
    Duomenu_aplankas = 'D:\DI\Data\MIT&ZIVE\VU\DUOM_VU'   # variantas: Windows
else:
    Duomenu_aplankas = '/home/kesju/DI/Data/MIT&ZIVE/VU/DUOM_VU'   # arba variantas: UBUNTU

# Nuoroda į aplanką su EKG įrašais (.npy) ir anotacijomis (.json)
rec_dir = Path(Duomenu_aplankas, 'records_npy')

# Failas pūpsnių klasių formavimui
all_beats =  {'N':0, 'S':1, 'V':2, 'U':3, 'F':3}  

# Diskretizavimo dažnis:
fs = 200

# Išvedame parametrus
print("\nBendras duomenų aplankas: ", Duomenu_aplankas)
print("Aplankas su originaliais EKG įrašais ir anotacijomis (.json) ", rec_dir)
print("Diskretizavimo dažnis: ", fs)
print('Klasifikavimo schema:', all_beats)


OS in my system :  win32

Bendras duomenų aplankas:  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU
Aplankas su originaliais EKG įrašais ir anotacijomis (.json)  D:\DI\Data\MIT&ZIVE\VU\DUOM_VU\records_npy
Diskretizavimo dažnis:  200
Klasifikavimo schema: {'N': 0, 'S': 1, 'V': 2, 'U': 3, 'F': 3}


In [16]:
# # Pagrindinis skriptas testavimui

# Užduodame pacientų EKG įrašų kodus

SubjCodes = [10000, 10001, 10020]

# Ciklas per įrašus

# Ciklas per pacientų įrašus
for SubjCode in SubjCodes:
    
    # Nuskaitome EKG įrašą (npy formatu)
    signal = read_rec(rec_dir, SubjCode)
    signal_length = signal.shape[0]

    # Surandame ir išvedame paciento ir įrašo kodus, EKG reikšmių kiekį
    userNr, recNr = split_SubjCode(SubjCode)
    print(f"\nSubjCode: {SubjCode} userNr: {userNr:>2} recNr: {recNr:>2} signal_length: {signal_length}")

    # Nuskaitome iš json failo EKG įrašo rpeaks indeksus ir atitinkamas anotacijas
    # (rpeaks: "sampleIndex" ir "annotationValue") 
    atr_sample, atr_symbol = read_rec_attrib(rec_dir, SubjCode)
    print(atr_sample[:10])
    print(atr_symbol[:10])

    # Iš anotacijų suformuojame klasių numerius
    # Jei pasitaiko symbol 'U' arba 'F', pūpsniui suteikiame klasę 3  
    class_labels = np.array([all_beats[symbol] for symbol in atr_symbol])

    # Apskaičiuojame ir išvedame pasiskirstymą per klases
    (unique, counts) = np.unique(class_labels, return_counts=True)
    total = counts.sum()
    print("class labels: ", unique, counts, total)



SubjCode: 10000 userNr: 1000 recNr:  0 signal_length: 127999
[ 166  333  506  681  853 1028 1205 1371 1530 1688]
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N']
class labels:  [0 1] [762   4] 766

SubjCode: 10001 userNr: 1000 recNr:  1 signal_length: 127999
[ 132  303  469  637  803  960 1120 1277 1433 1586]
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N']
class labels:  [0 1] [859   2] 861

SubjCode: 10020 userNr: 1002 recNr:  0 signal_length: 127999
[ 74 173 271 368 467 564 661 759 857 955]
['N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N' 'N']
class labels:  [0 1 2] [1166    2   17] 1185
